In [42]:
#Importing Deep Learnign Libraries
import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

#Importing Data Manipulation Libraries
import pandas as pd
import numpy as np

#Importing Data Preprocessing Libraries
import sklearn
from sklearn.preprocessing import Binarizer

#Import Path Libraries
import os
from os import listdir
from os.path import isfile, join

'''
Tensorflow Image Data import needs tf-nightly (nightly updated build) package: 
pip install tf-nightly
'''

'\nTensorflow Image Data import needs tf-nightly (nightly updated build) package: \npip install tf-nightly\n'

In [43]:
mypath = '/Users/jakob/OneDrive - Universitat Ramón Llull/TechLabsProj/cnn_input'

In [44]:
####-------------------- Dataset Import ------------------------------------####

In [45]:
## same day estimation  dataset ## 
y_train = pd.read_csv(f'{mypath}/y_train.csv')
y_test = pd.read_csv(f'{mypath}/y_test.csv')

In [46]:
y_train = y_train.values
y_test = y_test.values

In [47]:
## next day prediction  dataset ## 
y_train_pred = pd.read_csv(f'{mypath}/y_train_pred.csv')
y_test_pred = pd.read_csv(f'{mypath}/y_test_pred.csv')

In [48]:
y_train_pred = y_train_pred.values
y_test_pred = y_test_pred.values

In [49]:
####-------------------- Target Binarization ------------------------------------####

In [50]:
#binarize target (daily return) to 0 & 1 -> same day estimation
sklearn.preprocessing.Binarizer(threshold=0.0, copy=False)
y_train = Binarizer().fit_transform(y_train.reshape(1, -1))[0]
y_test = Binarizer().fit_transform(y_test.reshape(1, -1))[0]

In [51]:
#transform to integer
y_train = list(map(int, y_train))
y_test = list(map(int, y_test))

In [52]:
#binarize target (daily return) to 0 & 1 -> next day pred
y_train_pred = Binarizer().fit_transform(y_train_pred.reshape(1, -1))[0]
y_test_pred = Binarizer().fit_transform(y_test_pred.reshape(1, -1))[0]

In [53]:
#transform to integer
y_train_pred = list(map(int, y_train))
y_test_pred = list(map(int, y_test))

In [54]:
####-------------------- Creation of Train & Test Dataset for tracking input ------------------------------------####

In [55]:
## same day estimation dirs ##
'''
batch_size = 300
train_dir = f'{mypath}/pyplot_labelling/x_train
test_dir = f'{mypath}/pyplot_labelling/x_test
'''

"\nbatch_size = 300\ntrain_dir = f'{mypath}/pyplot_labelling/x_train\ntest_dir = f'{mypath}/pyplot_labelling/x_test\n"

In [57]:
## next day prediction dirs ##
batch_size = 300
train_dir = f'{mypath}/pyplot_labelling_prediction/x_train'
test_dir = f'{mypath}/pyplot_labelling_prediction/x_test'

In [58]:
x_train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(6, 6),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

Found 6247 files belonging to 2 classes.


In [59]:
x_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(6, 6),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

Found 2071 files belonging to 2 classes.


In [60]:
####-------------------- Deinition of CNN Architecture ------------------------------------####

In [61]:
def create_model():
    ## Initialize model.
    model = Sequential()

    ## 1st Conv Layer
    model.add(Conv2D(34, kernel_size=3, activation='relu', input_shape=(6, 6, 1), name='conv2_1', groups=0))
    
    ## 2nd Conv Layer + Pooling
    model.add(Conv2D(32, kernel_size=3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='max_pool_1'))

    ## Flatten for input in dense layers
    model.add(Flatten())
    
    #Adding 3 Dense Layers with final binary sigmoid
    model.add(Dense(32, activation='relu', name='dense_1'))
    model.add(Dense(8, activation='relu', name='dense_2'))
    model.add(Dense(2, activation='sigmoid', name='dense_3'))
    
    ## Define loss function
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [62]:
FNN = create_model()

In [1]:
####-------------------- Train CNN ------------------------------------####

In [63]:
FNN.fit(x_train, epochs=100, batch_size=batch_size, shuffle=True, verbose=2)

Epoch 1/100
21/21 - 6s - loss: 2.0751 - accuracy: 0.5036
Epoch 2/100
21/21 - 4s - loss: 0.7996 - accuracy: 0.5065
Epoch 3/100
21/21 - 4s - loss: 0.7135 - accuracy: 0.5118
Epoch 4/100
21/21 - 4s - loss: 0.7032 - accuracy: 0.5215
Epoch 5/100
21/21 - 4s - loss: 0.7027 - accuracy: 0.5137
Epoch 6/100
21/21 - 4s - loss: 0.7056 - accuracy: 0.5190
Epoch 7/100
21/21 - 4s - loss: 0.6993 - accuracy: 0.5231
Epoch 8/100
21/21 - 4s - loss: 0.7057 - accuracy: 0.5188
Epoch 9/100
21/21 - 4s - loss: 0.7064 - accuracy: 0.5191
Epoch 10/100
21/21 - 4s - loss: 0.7085 - accuracy: 0.5143
Epoch 11/100
21/21 - 4s - loss: 0.7015 - accuracy: 0.5188
Epoch 12/100
21/21 - 4s - loss: 0.6981 - accuracy: 0.5249
Epoch 13/100
21/21 - 4s - loss: 0.7004 - accuracy: 0.5202
Epoch 14/100
21/21 - 4s - loss: 0.6974 - accuracy: 0.5254
Epoch 15/100
21/21 - 4s - loss: 0.6997 - accuracy: 0.5273
Epoch 16/100
21/21 - 4s - loss: 0.7068 - accuracy: 0.5236
Epoch 17/100
21/21 - 4s - loss: 0.7069 - accuracy: 0.5188
Epoch 18/100
21/21 - 4s

KeyboardInterrupt: 

In [64]:
####-------------------- Evaluate CNN Performance ------------------------------------####

In [65]:
FNN.evaluate(x_test)

7/7 [==============================] - 0s 1ms/step - loss: 0.6952 - accuracy: 0.5307


[0.6951588988304138, 0.5306615233421326]

In [93]:
"""
Best so far (same day est):
23/23 [==============================] - 2s 83ms/step - loss: 0.3801 - accuracy: 0.8634
Batch Size: 100
Epochs = 100

23/23 [==============================] - 2s 86ms/step - loss: 0.6078 - accuracy: 0.6608
Batch Size: 100
Epochs = 10
"""

'\nBest so far:\nBatch Size: 100\nEpochs = 10\n'

In [ ]:
"""
Best so far (next day pred):
7/7 [==============================] - 0s 1ms/step - loss: 0.6952 - accuracy: 0.5307
Batch Size = 100
Epochs = 26
"""